In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
dataset="fmnist"
flag1=1 #train
flag2=1 #test

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
if dataset=="CIFAR-10":
   fp='/content/gdrive/My Drive/Colab Notebooks/CIFAR-10'
else:
  fp='/content/gdrive/My Drive/Colab Notebooks/Fashion-MNIST'
  
print(fp)
def swish(x):
  return x*tf.nn.sigmoid(x)

/content/gdrive/My Drive/Colab Notebooks/Fashion-MNIST


In [0]:


def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np
    path=fp

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [4]:
!pip install tsne


  Stored in directory: /root/.cache/pip/wheels/69/b4/26/9a673d2333de6af3865b5a2b462b3b0cfe4556ae69979434dd
Successfully built tsne


In [0]:
import pickle
import numpy as np
import os
from urllib.request import urlretrieve
import tarfile
import zipfile
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from time import time
import math
from sklearn.model_selection import train_test_split
from tsne import bh_sne

def get_data_set(fp,name="train"):
    x = None
    y = None

    if name is "train":
        for i in range(5):
            f = open(fp+'/data_batch_' + str(i + 1), 'rb')
            datadict = pickle.load(f, encoding='latin1')
            f.close()

            _X = datadict["data"]
            _Y = datadict['labels']

            _X = np.array(_X, dtype=float) / 255.0
            _X = _X.reshape([-1, 3, 32, 32])
            _X = _X.transpose([0, 2, 3, 1])
            _X = _X.reshape(-1, 32*32*3)

            if x is None:
                x = _X
                y = _Y
            else:
                x = np.concatenate((x, _X), axis=0)
                y = np.concatenate((y, _Y), axis=0)

    elif name is "test":
        f = open(fp+'/test_batch', 'rb')
        datadict = pickle.load(f, encoding='latin1')
        f.close()

        x = datadict["data"]
        y = np.array(datadict['labels'])

        x = np.array(x, dtype=float) / 255.0
        x = x.reshape([-1, 3, 32, 32])
        x = x.transpose([0, 2, 3, 1])
        x = x.reshape(-1, 32*32*3)
    #print(y)
    return x, y


def dense_to_one_hot(labels_dense, num_classes=10):
    #print(labels_dense.shape[0])
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1

    return labels_one_hot






In [0]:
def convl(input_,output_dim,kernal_size,activation_):
    conv = tf.layers.conv2d(
            inputs=input_,
            filters=output_dim,
            kernel_size=[kernal_size, kernal_size],kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
            padding='SAME',
            activation=activation_
        )
    pool= tf.layers.max_pooling2d(conv, pool_size=[2, 2], strides=2, padding='SAME')
    norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')
    return norm1
  
def model1(a,b,filter_list,activationfn):#a is image size,b is image channel
    _IMAGE_SIZE = a
    _IMAGE_CHANNELS = b
    _NUM_CLASSES = 10
    with tf.name_scope('main_params'):
        x = tf.placeholder(tf.float32, shape=[None, _IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS], name='Input')
        y = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='Output')
        x_image = tf.reshape(x, [-1, _IMAGE_SIZE, _IMAGE_SIZE, _IMAGE_CHANNELS], name='images')

        global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        learning_rate = tf.placeholder(tf.float32, shape=[], name='learning_rate')
    input_=x_image
    
    for i in range(len(filter_list)):
      a="conv"+str(i+1)
      with tf.variable_scope(a) as scope:
       
          x_filtersize=filter_list[i];
          conv=convl(input_,64*(i+1),x_filtersize,activationfn)
          drop = tf.layers.dropout(conv, rate=0.25, name=scope.name)
          input_=drop

    keep_prob=.75
    flat = tf.contrib.layers.flatten(drop)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=activationfn)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)

    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=activationfn)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)

    # 12
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=activationfn)
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)    

    # 13
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=1024, activation_fn=activationfn)
    full4 = tf.nn.dropout(full4, keep_prob)
    full4 = tf.layers.batch_normalization(full4)        

    #fc = tf.layers.dense(inputs=flat, units=1500, activation=tf.nn.relu)
    #drop = tf.layers.dropout(fc, rate=0.5)
    softmax = tf.layers.dense(inputs=full4, units=_NUM_CLASSES, name=scope.name)

    y_pred_cls = tf.argmax(softmax, axis=1)

    return x, y, softmax, y_pred_cls, global_step, learning_rate,full2
          
    
  

In [0]:


def lr(epoch):
    learning_rate = 1e-3
    if epoch > 80:
        learning_rate *= 0.5e-3
    elif epoch > 60:
        learning_rate *= 1e-4
    elif epoch > 40:
        learning_rate *= 1e-2
    elif epoch > 20:
        learning_rate *= 1e-1
    return learning_rate

In [8]:
#dataset="f"
#test_x, test_y = get_data_set("test")

tf.set_random_seed(21)
tf.reset_default_graph()
#just to reset the graph with same name
#x, y, output, y_pred_cls, global_step, learning_rate,full1 = model()
sess = tf.Session()
if dataset=="CIFAR-10":
     x, y, output, y_pred_cls, global_step, learning_rate,full2 = model1(32,3,[8,4,4],swish)
else:
     x, y, output, y_pred_cls, global_step, learning_rate,full2 = model1(28,1,[8,4,4],swish)
global_accuracy = 0
epoch_start = 0


# PARAMS
_BATCH_SIZE = 128
_EPOCH = 60
if dataset=="CIFAR-10":
   _SAVE_PATH ="/content/gdrive/My Drive/Colab Notebooks/model_cifar/CIFAR"
   checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/model_cifar/CIFAR"
else:
    _SAVE_PATH ="/content/gdrive/My Drive/Colab Notebooks/model_fmnist/FMINST"
    checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/model_fmnist"


# LOSS AND OPTIMIZER
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                   beta1=0.9,
                                   beta2=0.999,
                                   epsilon=1e-08).minimize(loss, global_step=global_step)


# PREDICTION AND ACCURACY CALCULATION
correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:






def train(sess,epoch,train_x,train_y,test_x,test_y):
    global epoch_start
    
    epoch_start = time()
    batch_size = int(math.ceil(len(train_x) / _BATCH_SIZE))
    i_global = 0

    for s in range(batch_size):
        batch_xs = train_x[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]
        batch_ys = train_y[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]

        start_time = time()
        i_global, _, batch_loss, batch_acc = sess.run(
            [global_step, optimizer, loss, accuracy],
            feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)})
        duration = time() - start_time

        if s % 10 == 0:
            percentage = int(round((s/batch_size)*100))

            bar_len = 29
            filled_len = int((bar_len*int(percentage))/100)
            bar = '=' * filled_len + '>' + '-' * (bar_len - filled_len)

            msg = "Global step: {:>5} - [{}] {:>3}% - acc: {:.4f} - loss: {:.4f} - {:.1f} sample/sec"
            print(msg.format(i_global, bar, percentage, batch_acc, batch_loss, _BATCH_SIZE / duration))

    return test_and_save(sess,i_global, epoch,test_x,test_y)


def test_and_save(sess,_global_step, epoch,test_x,test_y):
    global global_accuracy
    global epoch_start
    global F_micro
    global F_macro

    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(
            y_pred_cls,
            feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)}
        )
        i = j
    actual_=np.argmax(test_y, axis=1)
    correct = (np.argmax(test_y, axis=1) == predicted_class)
    
    acc = correct.mean()*100
    micro=f1_score(actual_,predicted_class, average='micro')
    macro=f1_score(actual_,predicted_class, average='macro')
    correct_numbers = correct.sum()

    hours, rem = divmod(time() - epoch_start, 3600)
    minutes, seconds = divmod(rem, 60)
    mes = "\nEpoch {} - accuracy: {:.2f}% ({}/{}) - time: {:0>2}:{:0>2}:{:05.2f}"
    print(mes.format((epoch+1), acc, correct_numbers, len(test_x), int(hours), int(minutes), seconds))
    train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)
    if global_accuracy != 0 and global_accuracy < acc:

        summary = tf.Summary(value=[
            tf.Summary.Value(tag="Accuracy/test", simple_value=acc),
        ])
        train_writer.add_summary(summary)

        saver.save(sess, save_path=_SAVE_PATH)

        mes = "This epoch receive better accuracy: {:.2f} > {:.2f}. Saving session..."
        print(mes.format(acc, global_accuracy))
        global_accuracy = acc
        F_micro=micro
        F_macro=macro

    elif global_accuracy == 0:
        global_accuracy = acc
        F_micro=micro
        F_macro=macro

    print("###########################################################################################################")
    return acc,micro,macro,global_accuracy,F_micro,F_macro



In [0]:
def test(sess,test_x,test_y):
        #test_x, test_y = get_data_set(fp,"test")
        #x, y, output, y_pred_cls, global_step, learning_rate = model()


        _BATCH_SIZE = test_x.shape[0]
        _CLASS_SIZE = 10
        #_SAVE_PATH ='/content/gdrive/My Drive/Colab Notebooks/CIFAR'


        saver = tf.train.Saver()



        #try:
        print("\nTrying to restore last checkpoint ...")
        #last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
        #print(_SAVE_PATH, last_chk_path)
        saver.restore(sess, save_path=_SAVE_PATH)
        #saver.restore(sess, save_path='%s-15614' % _SAVE_PATH)        
        #print("Restored checkpoint from:", '%s-15614' % _SAVE_PATH)
        #except ValueError:
            #print("\nFailed to restore checkpoint. Initializing variables instead.")
            #sess.run(tf.global_variables_initializer())


        def main():
            i = 0


            predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
           
            a,b = sess.run([y_pred_cls,full2], feed_dict={x: test_x, y: test_y})
            predicted_class=a
            actual_=np.argmax(test_y, axis=1)
            print(b.shape)
            n_sne = 5000
            b = np.asarray(b).astype('float64')
         
            f_tsne=bh_sne(b)
            plt.scatter(f_tsne[:,0], f_tsne[:,1], c=actual_, cmap=plt.cm.get_cmap("jet", 10))
            plt.colorbar(ticks=range(10))
            plt.clim(-0.5, 9.5)
            plt.show()
            plt.savefig('t_sne_1.png')

            from collections import defaultdict
            kmeans = KMeans(n_clusters=10, random_state=0).fit_transform(b)
            #print(kmeans[1][1])
            #ids_=np.arange(10)
            correct = 0
            label=defaultdict(list)
            for i in range(kmeans.shape[0]):
              label[np.argmax(kmeans[i])].append(actual_[i])
            z_=np.zeros(10,dtype=int)
            for i in range(10):
              z__=np.zeros(10,dtype=int)
              for j in range(len(label[i])):
                z__[label[i][j]]+=1
              z_[i]+=max(z__)
            print(np.sum(z_)/float(len(b)))
            
            
              
                
            
                #predict_me = predict_me.reshape(-1, len(predict_me))
            """
            for i in range(len(b)):
                predict_me = f_tsne[i]
                predict_me = predict_me.reshape(-1, len(predict_me))
                prediction = kmeans.predict(predict_me)
                x__=prediction[0]
                permu.append(x__)
              
            correct,correctlabel=maxacc(permu,actual_)
            print(correct)
            """

            print("f_tsne shape",f_tsne.shape)

            
            correct = (np.argmax(test_y, axis=1) == predicted_class)
            acc = correct.mean() * 100
            micro=f1_score(actual_,predicted_class, average='micro')
            macro=f1_score(actual_,predicted_class, average='macro')
            
            
            correct_numbers = correct.sum()
            print()
            print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc, correct_numbers, len(test_x)))
            print("MICRO ON TEST_SET",micro)
            print("MACRO ON TEST_SET",macro)



        if __name__ == "__main__":
            main()


        sess.close()
 

In [11]:
print(_SAVE_PATH)

/content/gdrive/My Drive/Colab Notebooks/model_fmnist/FMINST


In [0]:

def main():
    train_start = time()
    microl=[]
    accl=[]
    macrol=[]
    epochl=[]
    if flag1==1:
      if dataset=="CIFAR-10":
          train_x, train_y = get_data_set(fp,"train")
      else:
          train_x, train_y = load_mnist(fp,"train")
      train_x, test_x,train_y,test_y = train_test_split(train_x, train_y, test_size=0.1, random_state=42)
      test_y=dense_to_one_hot(test_y)
      train_y=dense_to_one_hot(train_y)
      #test(test_x,test_y)

      sess.run(tf.global_variables_initializer())
      for i in range(_EPOCH):
          print("\nEpoch: {}/{}\n".format((i+1), _EPOCH))
          acc,micro,macro,a,b,c=train(sess,i,train_x,train_y,test_x,test_y)
          print(acc,micro,macro)
          microl.append(micro)
          accl.append(acc)
          macrol.append(macro)
          epochl.append(i)
      plt.plot(epochl, microl, label='F1_micro')
      plt.show()
      plt.plot(epochl, macrol, label='F1_macro')
      plt.show()
      plt.plot(epochl, accl, label='accuracy')
      plt.show()
      hours, rem = divmod(time() - train_start, 3600)
      minutes, seconds = divmod(rem, 60)
      mes = "Best accuracy pre session: {:.2f}, time: {:0>2}:{:0>2}:{:05.2f}"
      print(mes.format(global_accuracy, int(hours), int(minutes), seconds))
    if flag2==1:
      if dataset=="CIFAR-10":
          train_x, train_y = get_data_set(fp,"train")
      else:
          train_x, train_y = load_mnist(fp,"train")
      test(sess,train_x,train_y)


if __name__ == "__main__":
    main()




Epoch: 1/60

Global step:     1 - [>-----------------------------]   0% - acc: 0.1016 - loss: 3.5268 - 41.1 sample/sec
Global step:    11 - [>-----------------------------]   2% - acc: 0.2969 - loss: 1.7570 - 2270.3 sample/sec
Global step:    21 - [=>----------------------------]   5% - acc: 0.3750 - loss: 1.3949 - 2282.6 sample/sec
Global step:    31 - [==>---------------------------]   7% - acc: 0.5312 - loss: 1.2109 - 2224.0 sample/sec
Global step:    41 - [==>---------------------------]   9% - acc: 0.5938 - loss: 1.0139 - 2178.7 sample/sec
Global step:    51 - [===>--------------------------]  12% - acc: 0.6641 - loss: 0.8992 - 2268.1 sample/sec
Global step:    61 - [====>-------------------------]  14% - acc: 0.6719 - loss: 0.8094 - 2286.6 sample/sec
Global step:    71 - [====>-------------------------]  17% - acc: 0.7812 - loss: 0.6369 - 2248.5 sample/sec
Global step:    81 - [=====>------------------------]  19% - acc: 0.6875 - loss: 0.8026 - 2300.9 sample/sec
Global step:    